# ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರ ಮಧ್ಯಸ್ಥ Middleware ಬಳಸಿ ಹೋಟೆಲ್ ಬುಕ್ಕಿಂಗ್

ಈ ನೋಟು Microsoft Agent Framework ಬಳಸಿ **ಕಾರ್ಯಾಧಾರಿತ ಮಧ್ಯಸ್ಥ Middleware** ಅನ್ನು ತೋರಿಸುತ್ತದೆ. ನಾವು ಶರತಾಯಿತ ಕಾರ್ಯಪ್ರವಾಹದ ಉದಾಹರಣೆಯನ್ನು ಅಭಿವೃದ್ಧಿಪಡಿಸುತ್ತೇವೆ,Middleware ಲೇಯರ್ ಅನ್ನು ಸೇರಿಸುವ ಮೂಲಕ ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರಿಗೆ ವಿಶೇಷ ಸೌಲಭ್ಯಗಳನ್ನು ನೀಡುತ್ತೇವೆ.

## ನೀವು ಕಲಿಯುವ ವಿಷಯಗಳು:
1. **ಕಾರ್ಯಾಧಾರಿತ Middleware**: ಕಾರ್ಯ ಫಲಿತಾಂಶಗಳನ್ನು ತಡೆದು, ಬದಲಾಯಿಸಿ
2. **ಸಂದರ್ಭ ಪ್ರವೇಶ**: ಕಾರ್ಯಾಚರಣೆಯ ನಂತರ `context.result` ಅನ್ನು ಓದಿ ಮತ್ತು ಬದಲಾಯಿಸಿ
3. **ವ್ಯಾಪಾರ ಲಾಜಿಕ್ ಅನುಷ್ಠಾನ**: ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರ ಲಾಭಗಳು
4. **ಫಲಿತಾಂಶ ಬದಲಾವಣೆ**: ಬಳಕೆದಾರರ ಸ್ಥಿತಿಯ ಆಧಾರದ ಮೇಲೆ ಕಾರ್ಯ ಫಲಿತಾಂಶಗಳನ್ನು ಬದಲಾಯಿಸಿ
5. **ಒಂದೇ ಕಾರ್ಯಪ್ರವಾಹ, ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳು**: Middleware-ಚಾಲಿತ ವರ್ತನೆ ಬದಲಾವಣೆಗಳು

## Middleware ಸಹಿತ ಕಾರ್ಯಪ್ರವಾಹದ ಆರ್ಕಿಟೆಕ್ಚರ್:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## ಶರತಾಯಿತ ಕಾರ್ಯಪ್ರವಾಹದಿಂದ ಪ್ರಮುಖ ವ್ಯತ್ಯಾಸ:

**Middleware ಇಲ್ಲದೆ** (14-conditional-workflow.ipynb):
- ಪ್ಯಾರಿಸ್‌ನಲ್ಲಿ ಕೊಠಡಿಗಳು ಇಲ್ಲ → alternative_agent ಗೆ ಮಾರ್ಗ

**Middleware ಸಹಿತ** (ಈ ನೋಟು):
- ಸಾಮಾನ್ಯ ಬಳಕೆದಾರ + ಪ್ಯಾರಿಸ್ → ಕೊಠಡಿಗಳು ಇಲ್ಲ → alternative_agent ಗೆ ಮಾರ್ಗ
- ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರ + ಪ್ಯಾರಿಸ್ → 🌟 Middleware ಬದಲಾವಣೆ! → ಲಭ್ಯವಿದೆ → booking_agent ಗೆ ಮಾರ್ಗ

## ಪೂರ್ವಶರತ್ತುಗಳು:
- Microsoft Agent Framework ಸ್ಥಾಪಿಸಲಾಗಿದೆ
- ಶರತಾಯಿತ ಕಾರ್ಯಪ್ರವಾಹಗಳ ಅರಿವು (14-conditional-workflow.ipynb ನೋಡಿ)
- GitHub ಟೋಕನ್ ಅಥವಾ OpenAI API ಕೀ
- Middleware ಮಾದರಿಗಳ ಮೂಲಭೂತ ಅರಿವು


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ಹಂತ 1: ರಚಿತ ಔಟ್‌ಪುಟ್‌ಗಳಿಗಾಗಿ ಪೈಡ್ಯಾಂಟಿಕ್ ಮಾದರಿಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿ

ಈ ಮಾದರಿಗಳು ಏಜೆಂಟ್‌ಗಳು ಹಿಂತಿರುಗಿಸುವ **schema** ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತವೆ. ಮಧ್ಯಸ್ಥಿಕೆ ಲಭ್ಯತೆಯ ಫಲಿತಾಂಶವನ್ನು ತಿದ್ದುಪಡಿ ಮಾಡಿದಾಗ ಟ್ರ್ಯಾಕ್ ಮಾಡಲು ನಾವು `priority_override` ಕ್ಷೇತ್ರವನ್ನು ಸೇರಿಸಿದ್ದೇವೆ.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ಹಂತ 2: ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರ ಡೇಟಾಬೇಸ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿ

ಈ ಡೆಮೊಗಾಗಿ, ನಾವು ಪ್ರಾಥಮಿಕ ಸದಸ್ಯತ್ವದ ಡೇಟಾಬೇಸ್ ಅನ್ನು ಅನುಕರಿಸುತ್ತೇವೆ. ಉತ್ಪಾದನೆಯಲ್ಲಿ, ಇದು ನಿಜವಾದ ಡೇಟಾಬೇಸ್ ಅಥವಾ API ಅನ್ನು ಕ್ವೆರಿ ಮಾಡುತ್ತದೆ.

**ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರು:**
- `alice@example.com` - VIP ಸದಸ್ಯ  
- `bob@example.com` - ಪ್ರೀಮಿಯಂ ಸದಸ್ಯ  
- `priority_user` - ಪರೀಕ್ಷಾ ಖಾತೆ  


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ಹಂತ 3: ಹೋಟೆಲ್ ಬುಕ್ಕಿಂಗ್ ಸಾಧನವನ್ನು ರಚಿಸಿ

ಶರತ್ತುಬದ್ಧ ಕಾರ್ಯಪ್ರವಾಹದಂತೆ, ಆದರೆ ಈಗ ಇದು ಮಧ್ಯವರ್ತಿಗಳಿಂದ ತಡೆಗಟ್ಟಲ್ಪಡುತ್ತದೆ!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ಹಂತ 4: 🌟 ಪ್ರಾಥಮಿಕತೆ ಪರಿಶೀಲನೆ ಮಿಡಲ್‌ವೇರ್ ರಚಿಸಿ (ಮುಖ್ಯ ವೈಶಿಷ್ಟ್ಯ!)

ಇದು ಈ ನೋಟ್ಬುಕ್‌ನ **ಮುಖ್ಯ ಕಾರ್ಯಕ್ಷಮತೆ**. ಮಿಡಲ್‌ವೇರ್:

1. **ಅಡಗಿಸುತ್ತದೆ** hotel_booking ಫಂಕ್ಷನ್ ಕರೆ
2. `next(context)` ಅನ್ನು ಕರೆದು ಫಂಕ್ಷನ್ ಅನ್ನು ಸಾಮಾನ್ಯವಾಗಿ **ನಿರ್ವಹಿಸುತ್ತದೆ**
3. `context.result` ನಲ್ಲಿ ಫಲಿತಾಂಶವನ್ನು **ಪರಿಶೀಲಿಸುತ್ತದೆ**
4. ಬಳಕೆದಾರನು ಪ್ರಾಥಮಿಕತೆಯಲ್ಲಿದ್ದರೆ ಮತ್ತು ಕೊಠಡಿಗಳು ಲಭ್ಯವಿಲ್ಲದಿದ್ದರೆ ಫಲಿತಾಂಶವನ್ನು **ಮರುಹೊಂದಿಸುತ್ತದೆ**
5. **ಮಾರ್ಪಡಿಸಿದ ಫಲಿತಾಂಶವನ್ನು** ಏಜೆಂಟ್‌ಗೆ ಹಿಂತಿರುಗಿಸುತ್ತದೆ

**ಮುಖ್ಯ ಮಾದರಿ:**
```python
async def my_middleware(context, next):
    await next(context)  # ಕಾರ್ಯವನ್ನು ನಿರ್ವಹಿಸಿ
    # ಈಗ context.result ನಲ್ಲಿ ಕಾರ್ಯದ ಔಟ್‌ಪುಟ್ ಇದೆ
    if some_condition:
        context.result = new_value  # ಮೀರಿಸಿ!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## ಹಂತ 6: ಕಸ್ಟಮ್ ಡಿಸ್ಪ್ಲೇ ಎಕ್ಸಿಕ್ಯೂಟರ್ ರಚಿಸಿ

ಹಿಂದಿನ ಎಕ್ಸಿಕ್ಯೂಟರ್‌ನಂತೆ - ಅಂತಿಮ ವರ್ಕ್‌ಫ್ಲೋ ಔಟ್‌ಪುಟ್ ಅನ್ನು ಪ್ರದರ್ಶಿಸುತ್ತದೆ.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ಹಂತ 7: ಪರಿಸರ ವ್ಯತ್ಯಾಸಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ

LLM ಕ್ಲೈಂಟ್ (GitHub Models ಅಥವಾ OpenAI) ಅನ್ನು ಸಂರಚಿಸಿ.


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ಹಂತ 8: ಮಧ್ಯವರ್ತಿತನದೊಂದಿಗೆ AI ಏಜೆಂಟ್‌ಗಳನ್ನು ರಚಿಸಿ

**ಮುಖ್ಯ ವ್ಯತ್ಯಾಸ:** availability_agent ರಚಿಸುವಾಗ, ನಾವು `middleware` ಪ್ಯಾರಾಮೀಟರ್ ಅನ್ನು ಪಾಸ್ ಮಾಡುತ್ತೇವೆ!

ಇದು priority_check_middleware ಅನ್ನು ಏಜೆಂಟ್‌ನ ಫಂಕ್ಷನ್ ಕರೆಯುವ ಪೈಪ್‌ಲೈನ್‌ಗೆ ಸೇರಿಸುವ ವಿಧಾನವಾಗಿದೆ.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ಹಂತ 9: ಕಾರ್ಯಪ್ರವಾಹವನ್ನು ನಿರ್ಮಿಸಿ

ಹಿಂದಿನಂತೆ ಕಾರ್ಯಪ್ರವಾಹದ ರಚನೆ - ಲಭ್ಯತೆಯ ಆಧಾರದ ಮೇಲೆ ಶರತಿನಿರ್ಣಯ ಮಾರ್ಗಸಂಚಾಲನೆ.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## ಹಂತ 10: ಪರೀಕ್ಷಾ ಪ್ರಕರಣ 1 - ಪ್ಯಾರಿಸ್‌ನ ಸಾಮಾನ್ಯ ಬಳಕೆದಾರ (ಓವರ್ರೈಡ್ ಇಲ್ಲ)

ಸಾಮಾನ್ಯ ಬಳಕೆದಾರನು ಪ್ಯಾರಿಸ್‌ನಲ್ಲಿ ಬುಕ್ ಮಾಡಲು ಪ್ರಯತ್ನಿಸುತ್ತಾನೆ → ಕೊಠಡಿಗಳು ಇಲ್ಲ → alternative_agent ಗೆ ಮಾರ್ಗಮಾಡುತ್ತದೆ


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## ಹಂತ 11: ಪರೀಕ್ಷಾ ಪ್ರಕರಣ 2 - 🌟 ಪ್ಯಾರಿಸ್‌ನಲ್ಲಿ ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರ (ಓವರ್‌ರೈಡ್ ಸಹಿತ!)

ಪ್ರಾಥಮಿಕ ಸದಸ್ಯನು ಪ್ಯಾರಿಸ್‌ನ್ನು ಬುಕ್ ಮಾಡಲು ಪ್ರಯತ್ನಿಸುತ್ತಾರೆ → ಪ್ರಾರಂಭದಲ್ಲಿ ಯಾವುದೇ ಕೊಠಡಿಗಳು ಇಲ್ಲ → 🌟 ಮಧ್ಯಸ್ಥ ಸಾಫ್ಟ್‌ವೇರ್ ಓವರ್‌ರೈಡ್ ಮಾಡುತ್ತದೆ! → booking_agent ಗೆ ಮಾರ್ಗಸೂಚಿ ಮಾಡುತ್ತದೆ

**ಇದು ಮಧ್ಯಸ್ಥ ಸಾಫ್ಟ್‌ವೇರ್ ಶಕ್ತಿಯ ಪ್ರಮುಖ ಪ್ರದರ್ಶನ!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ಹಂತ 12: ಪರೀಕ್ಷಾ ಪ್ರಕರಣ 3 - ಸ್ಟಾಕ್‌ಹೋಮ್‌ನಲ್ಲಿ ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರ (ಈಗಾಗಲೇ ಲಭ್ಯವಿದೆ)

ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರ ಸ್ಟಾಕ್‌ಹೋಮ್ ಪ್ರಯತ್ನಿಸುತ್ತಾರೆ → ಕೊಠಡಿಗಳು ಲಭ್ಯವಿದೆ → ಯಾವುದೇ ಓವರ್‌ರೈಡ್ ಅಗತ್ಯವಿಲ್ಲ → booking_agent ಗೆ ಮಾರ್ಗಮಾಡುತ್ತದೆ

ಇದು middleware ಕೇವಲ ಅಗತ್ಯವಿದ್ದಾಗ ಮಾತ್ರ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## ಮುಖ್ಯ ಅಂಶಗಳು ಮತ್ತು ಮಿಡಲ್‌ವೇರ್ ಪರಿಕಲ್ಪನೆಗಳು

### ✅ ನೀವು ಕಲಿತದ್ದೇನು:

#### **1. ಫಂಕ್ಷನ್ ಆಧಾರಿತ ಮಿಡಲ್‌ವೇರ್ ಮಾದರಿ**

ಮಿಡಲ್‌ವೇರ್ ಸರಳ async ಫಂಕ್ಷನ್ ಬಳಸಿ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ತಡೆಹಿಡಿಯುತ್ತದೆ:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # ಕಾರ್ಯಾಚರಣೆಯ ಮೊದಲು
    print("Intercepting...")
    
    # ಕಾರ್ಯಾಚರಣೆ ನಡೆಸಿ
    await next(context)
    
    # ಕಾರ್ಯಾಚರಣೆಯ ನಂತರ - ಫಲಿತಾಂಶ ಪರಿಶೀಲಿಸಿ
    if context.result:
        # ಅಗತ್ಯವಿದ್ದರೆ ಫಲಿತಾಂಶವನ್ನು ತಿದ್ದುಪಡಿ ಮಾಡಿ
        context.result = modified_value
```

#### **2. ಕಂಟೆಕ್ಸ್‌ಟ್ನ ಪ್ರವೇಶ ಮತ್ತು ಫಲಿತಾಂಶದ ಮರುಬದಲಾವಣೆ**

- `context.function` - ಕರೆಯಲಾಗುತ್ತಿರುವ ಫಂಕ್ಷನ್‌ನ್ನು ಪ್ರವೇಶಿಸಿ
- `context.arguments` - ಫಂಕ್ಷನ್ ಆರ್ಗ್ಯುಮೆಂಟ್‌ಗಳನ್ನು ಓದಿ
- `context.kwargs` - ಹೆಚ್ಚುವರಿ ಪ್ಯಾರಾಮೀಟರ್‌ಗಳನ್ನು ಪ್ರವೇಶಿಸಿ
- `await next(context)` - ಫಂಕ್ಷನ್‌ನ್ನು ಕಾರ್ಯಗತಗೊಳಿಸಿ
- `context.result` - ಫಂಕ್ಷನ್‌ನ ಔಟ್‌ಪುಟ್‌ನ್ನು ಓದಿ/ತಿದ್ದು

#### **3. ವ್ಯವಹಾರ ಲಾಜಿಕ್ ಅನುಷ್ಠಾನ**

ನಮ್ಮ ಮಿಡಲ್‌ವೇರ್ ಪ್ರಾಥಮಿಕ ಸದಸ್ಯರ ಪ್ರಯೋಜನಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುತ್ತದೆ:
- **ಸಾಮಾನ್ಯ ಬಳಕೆದಾರರು**: ಯಾವುದೇ ಬದಲಾವಣೆ ಇಲ್ಲ, ಸಾಮಾನ್ಯ ಕಾರ್ಯಪ್ರವಾಹ
- **ಪ್ರಾಥಮಿಕ ಬಳಕೆದಾರರು**: "ಲಭ್ಯತೆ ಇಲ್ಲ" → "ಲಭ್ಯವಿದೆ" ಎಂದು ಮರುಬದಲಾಯಿಸಿ
- **ಶರತ್ತು ಲಾಜಿಕ್**: ಅಗತ್ಯವಿದ್ದಾಗ ಮಾತ್ರ ಮರುಬದಲಾಯಿಸುತ್ತದೆ

#### **4. ಒಂದೇ ಕಾರ್ಯಪ್ರವಾಹ, ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳು**

ಮಿಡಲ್‌ವೇರ್‌ನ ಶಕ್ತಿ:
- ✅ ಕಾರ್ಯಪ್ರವಾಹದ ರಚನೆಗೆ ಯಾವುದೇ ಬದಲಾವಣೆ ಇಲ್ಲ
- ✅ ಟೂಲ್ ಫಂಕ್ಷನ್‌ಗೆ ಯಾವುದೇ ಬದಲಾವಣೆ ಇಲ್ಲ
- ✅ ಶರತ್ತು ಮಾರ್ಗದರ್ಶನ ಲಾಜಿಕ್‌ಗೆ ಯಾವುದೇ ಬದಲಾವಣೆ ಇಲ್ಲ
- ✅ ಕೇವಲ ಮಿಡಲ್‌ವೇರ್ → ವಿಭಿನ್ನ ವರ್ತನೆ!

### 🚀 ನಿಜಜೀವನದ ಅನ್ವಯಿಕೆಗಳು:

1. **VIP/ಪ್ರೀಮಿಯಂ ವೈಶಿಷ್ಟ್ಯಗಳು**
   - ಪ್ರೀಮಿಯಂ ಬಳಕೆದಾರರಿಗಾಗಿ ದರ ಮಿತಿಗಳನ್ನು ಮರುಬದಲಾಯಿಸಿ
   - ಸಂಪತ್ತಿಗೆ ಪ್ರಾಥಮಿಕ ಪ್ರವೇಶವನ್ನು ಒದಗಿಸಿ
   - ಪ್ರೀಮಿಯಂ ವೈಶಿಷ್ಟ್ಯಗಳನ್ನು ಡೈನಾಮಿಕ್ ಆಗಿ ಅನ್ಲಾಕ್ ಮಾಡಿ

2. **A/B ಪರೀಕ್ಷೆ**
   - ಬಳಕೆದಾರರನ್ನು ವಿಭಿನ್ನ ಅನುಷ್ಠಾನಗಳಿಗೆ ಮಾರ್ಗದರ್ಶಿಸಿ
   - ನಿರ್ದಿಷ್ಟ ಬಳಕೆದಾರರೊಂದಿಗೆ ಹೊಸ ವೈಶಿಷ್ಟ್ಯಗಳನ್ನು ಪರೀಕ್ಷಿಸಿ
   - ಹಂತ ಹಂತವಾಗಿ ವೈಶಿಷ್ಟ್ಯಗಳನ್ನು ಬಿಡುಗಡೆ ಮಾಡಿ

3. **ಭದ್ರತೆ ಮತ್ತು ಅನುಸರಣೆ**
   - ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ಆಡಿಟ್ ಮಾಡಿ
   - ಸಂವೇದನಶೀಲ ಕಾರ್ಯಗಳನ್ನು ತಡೆಹಿಡಿಯಿರಿ
   - ವ್ಯವಹಾರ ನಿಯಮಗಳನ್ನು ಜಾರಿಗೊಳಿಸಿ

4. **ಕಾರ್ಯಕ್ಷಮತೆಯ ಆಪ್ಟಿಮೈಸೇಶನ್**
   - ನಿರ್ದಿಷ್ಟ ಬಳಕೆದಾರರಿಗಾಗಿ ಫಲಿತಾಂಶಗಳನ್ನು ಕ್ಯಾಶ್ ಮಾಡಿ
   - ಸಾಧ್ಯವಾದಾಗ ದುಬಾರಿ ಕಾರ್ಯಗಳನ್ನು ಬಿಟ್ಟುಬಿಡಿ
   - ಡೈನಾಮಿಕ್ ಸಂಪತ್ತಿನ ಹಂಚಿಕೆ

5. **ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ಮರುಪ್ರಯತ್ನ**
   - ದೋಷಗಳನ್ನು ಸುಂದರವಾಗಿ ಹಿಡಿದು ನಿರ್ವಹಿಸಿ
   - ಮರುಪ್ರಯತ್ನ ಲಾಜಿಕ್ ಅನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸಿ
   - ಪರ್ಯಾಯ ಅನುಷ್ಠಾನಗಳಿಗೆ ಹಿಂತಿರುಗಿ

6. **ಲಾಗಿಂಗ್ ಮತ್ತು ಮಾನಿಟರಿಂಗ್**
   - ಫಂಕ್ಷನ್ ಕಾರ್ಯಗತಗತ ಸಮಯಗಳನ್ನು ಟ್ರ್ಯಾಕ್ ಮಾಡಿ
   - ಪ್ಯಾರಾಮೀಟರ್‌ಗಳು ಮತ್ತು ಫಲಿತಾಂಶಗಳನ್ನು ಲಾಗ್ ಮಾಡಿ
   - ಬಳಕೆ ಮಾದರಿಗಳನ್ನು ಮಾನಿಟರ್ ಮಾಡಿ

### 🔑 ಡೆಕೋರೇಟರ್‌ಗಳೊಂದಿಗೆ ಪ್ರಮುಖ ವ್ಯತ್ಯಾಸಗಳು:

| ವೈಶಿಷ್ಟ್ಯ | ಡೆಕೋರೇಟರ್ | ಮಿಡಲ್‌ವೇರ್ |
|---------|-----------|------------|
| **ವ್ಯಾಪ್ತಿ** | ಒಬ್ಬ ಫಂಕ್ಷನ್ | ಏಜೆಂಟ್‌ನ ಎಲ್ಲಾ ಫಂಕ್ಷನ್‌ಗಳು |
| **ಲಚೀಲತೆ** | ವ್ಯಾಖ್ಯಾನದಲ್ಲಿ ನಿಶ್ಚಿತ | ರನ್‌ಟೈಮ್‌ನಲ್ಲಿ ಡೈನಾಮಿಕ್ |
| **ಕಂಟೆಕ್ಸ್‌ಟ್** | ಸೀಮಿತ | ಸಂಪೂರ್ಣ ಏಜೆಂಟ್ ಕಂಟೆಕ್ಸ್‌ಟ್ |
| **ಸಂಯೋಜನೆ** | ಬಹು ಡೆಕೋರೇಟರ್‌ಗಳು | ಮಿಡಲ್‌ವೇರ್ ಪೈಪ್‌ಲೈನ್ |
| **ಏಜೆಂಟ್-ಅವೇರ್** | ಇಲ್ಲ | ಹೌದು (ಏಜೆಂಟ್ ಸ್ಥಿತಿಯನ್ನು ಪ್ರವೇಶಿಸಿ) |

### 📚 ಮಿಡಲ್‌ವೇರ್ ಅನ್ನು ಯಾವಾಗ ಬಳಸಬೇಕು:

✅ **ಮಿಡಲ್‌ವೇರ್ ಅನ್ನು ಬಳಸಿರಿ:**
- ಬಳಕೆದಾರ/ಸೆಷನ್ ಸ್ಥಿತಿಯ ಆಧಾರದ ಮೇಲೆ ವರ್ತನೆ ಬದಲಾಯಿಸಲು ನೀವು ಅಗತ್ಯವಿದೆ
- ನೀವು ಹಲವಾರು ಫಂಕ್ಷನ್‌ಗಳಿಗೆ ಲಾಜಿಕ್ ಅನ್ನು ಅನ್ವಯಿಸಲು ಬಯಸುತ್ತೀರಿ
- ನೀವು ಏಜೆಂಟ್ ಮಟ್ಟದ ಕಂಟೆಕ್ಸ್‌ಟ್‌ನ್ನು ಪ್ರವೇಶಿಸಲು ಅಗತ್ಯವಿದೆ
- ನೀವು ಲಾಗಿಂಗ್, ಪ್ರಾಮಾಣೀಕರಣ ಇತ್ಯಾದಿ ಕ್ರಾಸ್-ಕಟ್ಟಿಂಗ್ ಚಿಂತೆಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುತ್ತಿದ್ದೀರಿ

❌ **ಮಿಡಲ್‌ವೇರ್ ಅನ್ನು ಬಳಸಬೇಡಿ:**
- ಸರಳ ಇನ್‌ಪುಟ್ ಮಾನ್ಯತೆ (Pydantic ಅನ್ನು ಬಳಸಿ)
- ಫಂಕ್ಷನ್-ನಿರ್ದಿಷ್ಟ ಲಾಜಿಕ್ (ಫಂಕ್ಷನ್‌ನಲ್ಲಿ ಇರಿಸಿ)
- ಒಮ್ಮೆ ಮಾತ್ರ ಬದಲಾವಣೆಗಳು (ಫಂಕ್ಷನ್ ಅನ್ನು ಬದಲಾಯಿಸಿ)

### 🎓 ಪ್ರಗತಿಪೂರ್ವ ಮಾದರಿಗಳು:

```python
# ಬಹು ಮಧ್ಯವರ್ತಿಗಳು (ನಿರ್ವಹಣಾ ಕ್ರಮವು ಮುಖ್ಯವಾಗಿದೆ!)
middleware=[
    logging_middleware,      # ಮೊದಲು ಲಾಗ್‌ಗಳು
    auth_middleware,         # ನಂತರ ಪ್ರಾಮಾಣೀಕರಣವನ್ನು ಪರಿಶೀಲಿಸುತ್ತದೆ
    cache_middleware,        # ನಂತರ ಕ್ಯಾಶ್ ಅನ್ನು ಪರಿಶೀಲಿಸುತ್ತದೆ
    rate_limit_middleware,   # ನಂತರ ದರ ಮಿತಿಗಳನ್ನು ಹೊಂದಿಸುತ್ತದೆ
    priority_check_middleware  # ಕೊನೆಗೆ ಪ್ರಾಥಮಿಕತೆಯನ್ನು ಪರಿಶೀಲಿಸುತ್ತದೆ
]

# ಶರತಾಯಿತ ಮಧ್ಯವರ್ತಿ ಕಾರ್ಯಗತಗೊಳನೆ
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # ಫಲಿತಾಂಶವನ್ನು ತಿದ್ದು
    else:
        # ಸಂಪೂರ್ಣವಾಗಿ ಕಾರ್ಯಗತಗೊಳನೆಯನ್ನು ಬಿಟ್ಟುಬಿಡಿ
        context.result = cached_value
```

### 🔗 ಸಂಬಂಧಿತ ಪರಿಕಲ್ಪನೆಗಳು:

- **ಏಜೆಂಟ್ ಮಿಡಲ್‌ವೇರ್**: agent.run() ಕರೆಗಳನ್ನು ತಡೆಹಿಡಿಯುತ್ತದೆ
- **ಫಂಕ್ಷನ್ ಮಿಡಲ್‌ವೇರ್**: ಟೂಲ್ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ತಡೆಹಿಡಿಯುತ್ತದೆ (ನಾವು ಬಳಸಿದದ್ದು!)
- **ಮಿಡಲ್‌ವೇರ್ ಪೈಪ್‌ಲೈನ್**: ಕ್ರಮದಲ್ಲಿ ಕಾರ್ಯಗತಗತಗೊಳ್ಳುವ ಮಿಡಲ್‌ವೇರ್ ಸರಪಳಿ
- **ಕಂಟೆಕ್ಸ್‌ಟ್ ಪ್ರೊಪಗೇಶನ್**: ಮಿಡಲ್‌ವೇರ್ ಸರಪಳಿಯ ಮೂಲಕ ಸ್ಥಿತಿಯನ್ನು ಪಾಸ್ ಮಾಡಿ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಖಚಿತತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗಿರುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
